#1.Load the data from Github

In [ ]:
#Import Libraries 
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, f1_score

In [ ]:
#Google Drive Access Setup
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd 

# test_input = 'file:///content/drive/MyDrive/test_without_labels.csv'
# train_input = 'file:///content/drive/MyDrive/train.csv'
# valid_input = 'file:///content/drive/MyDrive/val.csv'

# id = '15ytwyux4X_hfeo9rKc54mScR29UOG6BL' 
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('sample.csv')

id = '186aUyJjlJx8tOKG1_Eli41_fQ4C9wDCi'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')

id = '1Gns3_mTVRSCKCvzTvCWrlI8HJ0XMyBHD'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('val.csv')

id = '11L-S5S3cLFE7pM3RVohFS9P5XX9QzMqe'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_without_labels.csv')

train = pd.read_csv('/content/train.csv')
val = pd.read_csv('/content/val.csv')
test = pd.read_csv('/content/test_without_labels.csv')

print("------------------------------------")
print("Size of training dataset: {0}".format(len(train)))
print("Size of validation dataset: {0}".format(len(val)))
print("Size of testing dataset: {0}".format(len(test)))
print("------------------------------------")
train.head(30)

------------------------------------
Size of training dataset: 26078
Size of validation dataset: 8705
Size of testing dataset: 500
------------------------------------


,sents,labels
0,wow,O
1,WTF,T
2,wpe wpe,O O
3,hahaha,O
4,wtf,T
5,i cant [SEPA] play [SEPA] with 4 trash,P O SEPA O SEPA O O O
6,bg,O
7,#ERROR!,O
8,gg [SEPA] report my team rat [SEPA] please,S SEPA S P O S SEPA O
9,ez mid,S S


In [ ]:
val.head()

In [ ]:
test.head()

In [ ]:
# Extract the sents and labels and store into List

# Get the list of training data (sents)
train_sents = list(train['sents'])
# Get the list of corresponding labels for the training data (sents)
train_labels = list(train['labels'])


# Get the list of validation data (sents)
val_sents = list(val['sents'])
# Get the list of corresponding labels for the validation data (sents)
val_labels = list(val['labels'])


# Get the list of testing data (sents)
test_sents = list(test['sents'])



In [ ]:
train_sents[:10]

# 2.Input Embedding

###Word Embedding 

In [ ]:
#Using pre-trainded embedding 
import numpy as np 
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25") 

# Embedding dimension
EMBEDDING_DIM = 25

word_to_ix = {}
for sentence in train_sents + val_sents + test_sents:
    for word in sentence.split():
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())
ix_to_word = {v:k for k,v in word_to_ix.items()}

# How many unique words
VOCAB_SIZE = len(word_to_ix)

# Addtional Words
START_TAG = "<START>"
STOP_TAG = "<STOP>"
SEPA = "sepa"
tag_to_ix = {START_TAG:0, STOP_TAG:1, SEPA:3}
for tags in train_labels+val_labels:
    for tag in tags:
      tag = tag.lower()
      if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)

# Create a Matrix: word -> vector
embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.self[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape


[==================================================] 100.0% 104.8/104.8MB downloaded


(11243, 25)

In [ ]:
def to_index(data, to_ix, lower=True):
    input = []
    for sent in data:
        input.append([to_ix[w.lower() if lower else w] for w in sent.split()])
    return input
train_input_index =  to_index(train_sents,word_to_ix)
train_output_index = to_index(train_labels,tag_to_ix)
val_input_index = to_index(val_sents,word_to_ix)
val_output_index = to_index(val_labels,tag_to_ix)
test_input_index = to_index(test_sents,word_to_ix)

# print('Input: ', len(train_input_index[1]))
# print('Output: ', len(train_output_index[1]))
print(len(train_input_index))
# print(len(train_output_index[5]))

26078


###Pos Tagging & Dependency Path

In [ ]:
import spacy
# Load your usual SpaCy model (one of SpaCy English models)
nlp = spacy.load('en_core_web_sm')


In [ ]:
#Generate the part of speech tag for the training data

def getpos_dep(sents):
  p = []
  d = []
  for s in sents:
    doc = nlp(s)
    p_temp = []
    d_temp = []
    for token in doc:
      p_temp.append(token.tag_)
      d_temp.append(token.dep_)
    p.append(p_temp)
    d.append(d_temp)
  return p,d

In [ ]:
#Convert the pos tagging, dependancy tagging into index
pos_idx = {}
dep_idx = {}

train_pos, train_dep = getpos_dep(train_sents)
val_pos, valA_dep = getpos_dep(val_sents)
test_pos, test_dep = getpos_dep(test_sents)

In [ ]:
for ps in train_pos + val_pos + test_pos:
  for p in ps:
    if p not in pos_idx:
      pos_idx[p] = len(pos_idx)

for des in train_dep + valA_dep + test_dep:
  for de in des:
    if de not in dep_idx:
      dep_idx[de] = len(dep_idx)

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag

# Total number of PoS tags
POSTAG_SIZE = len(list(pos_idx.keys()))
postag_list = list(pos_idx.keys())

# PoS tag <--> word index
pos_to_ix = { k:v for v, k in enumerate(postag_list) }

# PoS Embedding Matrix
pos_embedding_matrix = np.zeros((VOCAB_SIZE, POSTAG_SIZE))
for word_idx, word in enumerate(word_list):
    tag = pos_tag([word])
    pos_idx = pos_to_ix[tag[0][1]]
    pos_embedding_matrix[word_idx, pos_idx] = 1

print(f'PoS Embedding Matrix: {pos_embedding_matrix.shape}\n{pos_embedding_matrix}')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
PoS Embedding Matrix: (11243, 49)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# 3.Slot Filling/Tagging Model

####CRF

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
#Reference from Lab code
class CRF_layer(torch.nn.Module):
    '''
    Claim: The CRF_layer Code is derived from PyTorch Tutorial. We wrote a wrapper class for it. 
    More: https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html
    '''
    def __init__(self, tag_to_ix):
        """
        """
        super(CRF_layer, self).__init__()

        self.vocab_size = VOCAB_SIZE
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        
    def forward(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(self._log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = self._log_sum_exp(terminal_var)
        return alpha

    def decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = self._argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = self._argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path
        
    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _argmax(self, vec):
        # return the argmax as a python int
        _, idx = torch.max(vec, 1)
        return idx.item()

    # Compute log sum exp in a numerically stable way for the forward algorithm
    def _log_sum_exp(self, vec):
        max_score = vec[0, self._argmax(vec)]
        max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
        return max_score + \
            torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

###Attention

In [ ]:
#Create the attention layer 
class Attention_layer(torch.nn.Module):
    def __init__(self, input_size, attention_type='dot', dropout_rate=0.5):
        """
        Define the trainable parameters for attention calculation.
        Inputs:
            inputsize:          input dimension size 
            attention_type:     dot | scaled_dot | cos_sim
        """
        super(Attention_layer, self).__init__()
        self.attention_type = attention_type

        self.K = nn.Linear(input_size, input_size, bias=False)
        self.Q = nn.Linear(input_size, input_size, bias=False)
        self.V = nn.Linear(input_size, input_size, bias=False)

        self.K_dropout = nn.Dropout(p=dropout_rate)
        self.Q_dropout = nn.Dropout(p=dropout_rate)
        self.V_dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        """
        Attention calculation
        For self-attention, the Key, Query and Value all equal to x
        """
        out_key = self.K_dropout(self.K(x))
        out_query = self.Q_dropout(self.Q(x))
        out_value = self.V_dropout(self.V(x))

        # Calculate the score in different ways 
        s = 0
        if self.attention_type == 'dot':
            s = torch.bmm(out_key, out_query.view(x.shape[0], -1, 1))
        elif self.attention_type == 'scaled_dot':
            s = torch.bmm(out_key, out_query.view(x.shape[0], -1, 1)) / torch.sqrt(torch.tensor(x.shape[-1]))
        elif self.attention_type == 'cos_sim':
            s = torch.bmm(out_key, out_query.view(x.shape[0], -1, 1)) / (torch.norm(out_key, p=2) * torch.norm(out_query, p=2))

        # Normalise the score 
        w = F.softmax(s, dim=0)

        # Matmul Score and Value
        y = torch.bmm(w, out_value)

        return y

###Model(Baseline with CRF)

In [ ]:
#Baseline model 
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)
        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)
        return feats

    # Foward Probagation
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        # Find the best path, given the features.
        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq

In [ ]:
# Calculate f1, predicted, ground_truth, accuracy scores
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1

In [ ]:
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


#Evaluate the result 
def evaluate_model(model):
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'ground truth: {y_true_decode[:20]}')
    print(f'prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch):
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log

In [ ]:
#LSTM
HIDDEN_DIM = 50                # the number of hidden dimension on LSTM
HIDDEN_LAYER = 1                # the number of hidden layer of LSTM

# Attention
ATTENTION_LAYER = 1             # the number of attention layer
ATTENTION_TYPE = 'scaled_dot'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_base = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_base.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_base}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_base, log = define_train_evaluate_model(model_base,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (lstm): LSTM(25, 25, bidirectional=True)
  (crf_layer): CRF_layer()
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 118108.62, train acc: 0.6750, train f1: 0.5440, val loss: 34559.88, val acc: 0.6772, val f1: 0.5469, time: 678.09s
Epoch:2, Training loss: 81955.59, train acc: 0.8156, train f1: 0.7634, val loss: 18629.69, val acc: 0.8164, val f1: 0.7642, time: 688.29s
Sample ground truth: ['s', 's', 's', 'o', 'o', 'o', 'o', 't', 'o', 'o', 'p', 'o', 'o', 'p', 'o', 't', 'o', 't', 'c', 'o']
Sample prediction: ['s', 's', 'p', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'p', 'o', 'o', 'p', 'o', 'o', 'o', 'o', 'o', 'o']
              precision    recall  f1-score   supp

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model(Word2Vec+Bi-LSTM + CRF + Attention(Scaled Dot))

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        
        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq

def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1


def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


#Sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log
# LSTM
HIDDEN_DIM = 50                # the number of hidden dimension on LSTM
HIDDEN_LAYER = 1                # the number of hidden layer of LSTM

# Attention
ATTENTION_LAYER = 1             # the number of attention layer
ATTENTION_TYPE = 'scaled_dot'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_wv_ls_crf_att = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_wv_ls_crf_att.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_wv_ls_crf_att}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_wv_ls_crf_att, log = define_train_evaluate_model(model_wv_ls_crf_att,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (lstm): LSTM(25, 25, bidirectional=True)
  (attention_layers): ModuleList(
    (0): Attention_layer(
      (K): Linear(in_features=50, out_features=50, bias=False)
      (Q): Linear(in_features=50, out_features=50, bias=False)
      (V): Linear(in_features=50, out_features=50, bias=False)
      (K_dropout): Dropout(p=0.0, inplace=False)
      (Q_dropout): Dropout(p=0.0, inplace=False)
      (V_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (crf_layer): CRF_layer()
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 118322.22, train acc: 0.6750, train f1: 0.5440, val loss: 35182.16, val acc: 0.6772, val f1: 0.5469, time: 695.90s
Epoch:2, Training loss: 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model(Stacked layers(2) Bi-LSTM + Word2Vec + CRF + Attention(dot))

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Pos Embeddings
        # self.pos_embeds = nn.Embedding(VOCAB_SIZE, POSTAG_SIZE)
        # self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding_matrix))
        # self.pos_linear = nn.Linear(POSTAG_SIZE, POSTAG_SIZE)

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)
        
        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # PoS Tag Embeddings
        # pos_embeds = self.pos_embeds(sentence).view(len(sentence), 1, -1)
        # pos_embeds = self.pos_linear(pos_embeds)

        # Concated Embeddings
        # embeds = torch.cat((word_embeds, pos_embeds), 2)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    # Use for inference
    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        # Find the best path, given the features.
        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq
# Calculate the scores given ground truth and prediction
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1



# Map the tag index to NER tag name
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


# Print out the prediction sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log
# LSTM
HIDDEN_DIM = 50               
HIDDEN_LAYER = 2                

# Attention
ATTENTION_LAYER = 1             
ATTENTION_TYPE = 'scaled_dot'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_lstm2 = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_lstm2.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_lstm2}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_lstm2, log = define_train_evaluate_model(model_lstm2,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (lstm): LSTM(25, 25, num_layers=2, bidirectional=True)
  (attention_layers): ModuleList(
    (0): Attention_layer(
      (K): Linear(in_features=50, out_features=50, bias=False)
      (Q): Linear(in_features=50, out_features=50, bias=False)
      (V): Linear(in_features=50, out_features=50, bias=False)
      (K_dropout): Dropout(p=0.0, inplace=False)
      (Q_dropout): Dropout(p=0.0, inplace=False)
      (V_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (crf_layer): CRF_layer()
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 115341.11, train acc: 0.6750, train f1: 0.5440, val loss: 35643.97, val acc: 0.6772, val f1: 0.5469, time: 765.91s
Epoch:2, T

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model(Word2Vec + Bi-LSTM + CRF + Attention(dot)

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Pos Embeddings
        # self.pos_embeds = nn.Embedding(VOCAB_SIZE, POSTAG_SIZE)
        # self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding_matrix))
        # self.pos_linear = nn.Linear(POSTAG_SIZE, POSTAG_SIZE)

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # PoS Tag Embeddings
        # pos_embeds = self.pos_embeds(sentence).view(len(sentence), 1, -1)
        # pos_embeds = self.pos_linear(pos_embeds)

        # Concated Embeddings
        # embeds = torch.cat((word_embeds, pos_embeds), 2)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    # Use for inference
    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        # Find the best path, given the features.
        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq
# Calculate the scores given ground truth and prediction
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1



# Map the tag index to NER tag name
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


# Print out the prediction sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log
# LSTM
HIDDEN_DIM = 50                # the number of hidden dimension on LSTM
HIDDEN_LAYER = 1                # the number of hidden layer of LSTM

# Attention
ATTENTION_LAYER = 1             # the number of attention layer
ATTENTION_TYPE = 'dot'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_att_dot = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_att_dot.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_att_dot}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_att_dot, log = define_train_evaluate_model(model_att_dot,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (lstm): LSTM(25, 25, bidirectional=True)
  (attention_layers): ModuleList(
    (0): Attention_layer(
      (K): Linear(in_features=50, out_features=50, bias=False)
      (Q): Linear(in_features=50, out_features=50, bias=False)
      (V): Linear(in_features=50, out_features=50, bias=False)
      (K_dropout): Dropout(p=0.0, inplace=False)
      (Q_dropout): Dropout(p=0.0, inplace=False)
      (V_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (crf_layer): CRF_layer()
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 114819.33, train acc: 0.6750, train f1: 0.5440, val loss: 35316.66, val acc: 0.6772, val f1: 0.5469, time: 754.45s
Epoch:2, Training loss: 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Print out the result 
model_att_dot.eval()
with torch.no_grad():
    predicted = []

    for i,idxs in enumerate(test_input_index):
        score, pred = model_att_dot(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
        
test_pred_decode = decode_output(predicted)
id = []
for i,item in enumerate(test_pred_decode):
    id.append(i)

df = pd.DataFrame({'ID':id,'Predicted':test_pred_decode})
df.to_csv(r"result_final.csv",index=False)

###Model(Word2Vec + Bi-LSTM + CRF + Attention(cos similarity)

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Pos Embeddings
        # self.pos_embeds = nn.Embedding(VOCAB_SIZE, POSTAG_SIZE)
        # self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding_matrix))
        # self.pos_linear = nn.Linear(POSTAG_SIZE, POSTAG_SIZE)

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # PoS Tag Embeddings
        # pos_embeds = self.pos_embeds(sentence).view(len(sentence), 1, -1)
        # pos_embeds = self.pos_linear(pos_embeds)

        # Concated Embeddings
        # embeds = torch.cat((word_embeds, pos_embeds), 2)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    # Use for inference
    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        # Find the best path, given the features.
        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq
# Calculate the scores given ground truth and prediction
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1



# Map the tag index to NER tag name
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


# Print out the prediction sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log
# LSTM
HIDDEN_DIM = 50                # the number of hidden dimension on LSTM
HIDDEN_LAYER = 1                # the number of hidden layer of LSTM

# Attention
ATTENTION_LAYER = 1             # the number of attention layer
ATTENTION_TYPE = 'cos_sim'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_att_cosm = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_att_cosm.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_att_cosm}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_att_cosm, log = define_train_evaluate_model(model_att_cosm,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (lstm): LSTM(25, 25, bidirectional=True)
  (attention_layers): ModuleList(
    (0): Attention_layer(
      (K): Linear(in_features=50, out_features=50, bias=False)
      (Q): Linear(in_features=50, out_features=50, bias=False)
      (V): Linear(in_features=50, out_features=50, bias=False)
      (K_dropout): Dropout(p=0.0, inplace=False)
      (Q_dropout): Dropout(p=0.0, inplace=False)
      (V_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (crf_layer): CRF_layer()
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 119679.53, train acc: 0.6750, train f1: 0.5440, val loss: 35794.57, val acc: 0.6772, val f1: 0.5469, time: 697.85s
Epoch:2, Training loss: 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model(Pos Tagging Embedding + Word2Vec + Bi-LSTM + CRF + Attention(Scaled Dot))

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Pos Embeddings
        self.pos_embeds = nn.Embedding(VOCAB_SIZE, POSTAG_SIZE)
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding_matrix))
        self.pos_linear = nn.Linear(POSTAG_SIZE, POSTAG_SIZE)

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # PoS Tag Embeddings
        pos_embeds = self.pos_embeds(sentence).view(len(sentence), 1, -1)
        pos_embeds = self.pos_linear(pos_embeds)

        # Concated Embeddings
        embeds = torch.cat((word_embeds, pos_embeds), 2)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    # Use for inference
    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        # Find the best path, given the features.
        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq
# Calculate the scores given ground truth and prediction
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1



# Map the tag index to NER tag name
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


# Print out the prediction sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log
# LSTM
HIDDEN_DIM = 50                # the number of hidden dimension on LSTM
HIDDEN_LAYER = 1                # the number of hidden layer of LSTM

# Attention
ATTENTION_LAYER = 1             # the number of attention layer
ATTENTION_TYPE = 'scaled_dot'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_attn_scaledot = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_attn_scaledot.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_attn_scaledot}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_attn_scaledot, log = define_train_evaluate_model(model_attn_scaledot,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (pos_embeds): Embedding(11243, 49)
  (pos_linear): Linear(in_features=49, out_features=49, bias=True)
  (lstm): LSTM(25, 25, bidirectional=True)
  (attention_layers): ModuleList(
    (0): Attention_layer(
      (K): Linear(in_features=50, out_features=50, bias=False)
      (Q): Linear(in_features=50, out_features=50, bias=False)
      (V): Linear(in_features=50, out_features=50, bias=False)
      (K_dropout): Dropout(p=0.0, inplace=False)
      (Q_dropout): Dropout(p=0.0, inplace=False)
      (V_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (crf_layer): CRF_layer()
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 114532.29, train acc: 0.6750, train

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model(Pos + Dependency + Word Embedding + Bi-LSTM + CRF + Attention(Scaled Dot))

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Pos Embeddings
        self.pos_embeds = nn.Embedding(VOCAB_SIZE, POSTAG_SIZE)
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding_matrix))
        self.pos_linear = nn.Linear(POSTAG_SIZE, POSTAG_SIZE)

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # PoS Tag Embeddings
        pos_embeds = self.pos_embeds(sentence).view(len(sentence), 1, -1)
        pos_embeds = self.pos_linear(pos_embeds)

        # Concated Embeddings
        embeds = torch.cat((word_embeds, pos_embeds), 2)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        forward_score = self.crf_layer.forward(feats)

        # gold_score = self._score_sentence(feats, tags)
        gold_score = self.crf_layer._score_sentence(feats, tags)

        return forward_score - gold_score

    # Use for inference
    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)

        # Find the best path, given the features.
        score, tag_seq = self.crf_layer.decode(feats)
        return score, tag_seq
# Calculate the scores given ground truth and prediction
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1



# Map the tag index to NER tag name
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


# Print out the prediction sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(y_pred)

    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')

    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    evaluate_model(model)

    return model, log
# LSTM
HIDDEN_DIM = 50                # the number of hidden dimension on LSTM
HIDDEN_LAYER = 1                # the number of hidden layer of LSTM

# Attention
ATTENTION_LAYER = 1             # the number of attention layer
ATTENTION_TYPE = 'scaled_dot'   # dot | scaled_dot | cos_sim

# Dropout rate on LSTM layers and Attention units
DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_attn_scaledot = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_attn_scaledot.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_attn_scaledot}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_attn_scaledot, log = define_train_evaluate_model(model_attn_scaledot,
                         num_epoch=NUM_EPOCH)

###Model(Word2Vec + Bi-LSTM + Attention(Scaled Dot) + Without CRF)

In [ ]:
class Model(nn.Module):
    def __init__(self, tag_to_ix, embedding_dim, hidden_dim, hidden_layer, dropout_rate, attention_layer, attention_type):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tag_to_ix)
        self.attention_layer = attention_layer

        # Word Embeddings 
        self.word_embeds = nn.Embedding(VOCAB_SIZE, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        # Pos Embeddings
        # self.pos_embeds = nn.Embedding(VOCAB_SIZE, POSTAG_SIZE)
        # self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding_matrix))
        # self.pos_linear = nn.Linear(POSTAG_SIZE, POSTAG_SIZE)

        # Bi-LSTM Layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, dropout=dropout_rate,
                            num_layers=hidden_layer, bidirectional=True)

        # Attention Layers
        self.attention_layers = nn.ModuleList()
        for layer in range(attention_layer):
            self.attention_layers.append(Attention_layer(embedding_dim+embedding_dim, attention_type, dropout_rate))

        # CRF Layer 
        # self.crf_layer = CRF_layer(tag_to_ix)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        self.ce = nn.CrossEntropyLoss()

    # Forward pass without CRF layer
    def _forward_pass(self, sentence):
        # Word Embeddings
        word_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        # PoS Tag Embeddings
        # pos_embeds = self.pos_embeds(sentence).view(len(sentence), 1, -1)
        # pos_embeds = self.pos_linear(pos_embeds)

        # Concated Embeddings
        # embeds = torch.cat((word_embeds, pos_embeds), 2)

        # LSTM Layers
        lstm_out, _ = self.lstm(word_embeds)

        # Attention Calculation
        attention_score = lstm_out
        for layer in self.attention_layers[:-1]:
            attention_score = layer(attention_score)

        att_out = lstm_out.view(len(sentence), self.hidden_dim)
        feats = self.hidden2tag(att_out)

        return feats

    # Use where Forward Propogation during training
    def forward_training(self, sentence, tags):
        feats = self._forward_pass(sentence)
        tag_seq = torch.argmax(feats, dim=1)
        score = self.ce(feats, tags)

        return score

    # Use for inference
    def forward(self, sentence): 
        # Get the emission scores from the BiLSTM
        feats = self._forward_pass(sentence)
        tag_seq = torch.argmax(feats, dim=1)
        score = self.ce(feats, tag_seq)

        return score, tag_seq
# Calculate the scores given ground truth and prediction
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        # print(pred)
        # print(pred.numpy())
        predicted += pred.cpu()
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='weighted')
    return predicted, ground_truth, accuracy, f1


# Map the tag index to NER tag name
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]


# Print out the prediction sample and classification report
def evaluate_model(model):
    # Get the results
    y_pred, y_true, _, _ = cal_acc(model, val_input_index, val_output_index)
    y_true_decode = decode_output(y_true)
    y_pred_decode = decode_output(list(y_pred))
    print(f'Sample ground truth: {y_true_decode[:20]}')
    print(f'Sample prediction: {y_pred_decode[:20]}')
    print(classification_report(y_true_decode, y_pred_decode, digits=4))


def define_train_evaluate_model(model, num_epoch=10):
    # For recording the training log for visualisation
    log = {
        'train_loss': [],
        'train_acc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': [],
    }

    # Training Loop
    for epoch in range(num_epoch):  
        time1 = datetime.datetime.now()
        train_loss = 0

        model.train()
        # Mini-Batch Training
        for i, idxs in enumerate(train_input_index):
            tags_index = train_output_index[i]

            # Clear the gradients
            model.zero_grad()

            # Prepare the input and output data as tensors 
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)

            # Forward propogation
            loss = model.forward_training(sentence_in, targets)

            # Backward Propogation
            loss.backward()

            # Update the model
            optimizer.step()

            # Accumulate the loss
            train_loss+=loss.item()

        # Evaluate the model on train_set and val_set
        model.eval()
        _, _, train_acc, train_f1 = cal_acc(model,train_input_index,train_output_index)
        _, _, val_acc, val_f1 = cal_acc(model,val_input_index,val_output_index)

        # Evaluate on the validation dataset 
        val_loss = 0
        for i, idxs in enumerate(val_input_index):
            tags_index = val_output_index[i]
            sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
            targets = torch.tensor(tags_index, dtype=torch.long).to(device)
            loss = model.forward_training(sentence_in, targets)
            val_loss+=loss.item()
        time2 = datetime.datetime.now()

        # The log of each epoch
        print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" \
            %(epoch+1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
        # Log the training log of each epoch
        log['train_loss'].append(train_loss)
        log['train_acc'].append(train_acc)
        log['train_f1'].append(val_acc)
        log['val_loss'].append(val_loss)
        log['val_acc'].append(val_acc)
        log['val_f1'].append(val_acc)

    # Evaluate the model
    # evaluate_model(model)
    return model, log
# LSTM
HIDDEN_DIM = 50                
HIDDEN_LAYER = 1                

# Attention
ATTENTION_LAYER = 1             
ATTENTION_TYPE = 'scaled_dot'   

DROPOUT_RATE = 0.0

# Training
NUM_EPOCH = 2
LR = 0.001
WEIGHT_DECAY = 1e-4

# Create the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_nocrf = Model(tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYER, DROPOUT_RATE, ATTENTION_LAYER, ATTENTION_TYPE).to(device)
optimizer = optim.SGD(model_nocrf.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
print(f'Model Setup: \n{model_nocrf}\n')
print(f'Optimizer Setup: \n{optimizer}\n')
print(f'------ Start Training, Testing, Evaluating Process | Total Epoch: {NUM_EPOCH} ------')

# Start Training, testing, evaluating process
model_nocrf, log = define_train_evaluate_model(model_nocrf,
                         num_epoch=NUM_EPOCH)

Model Setup: 
Model(
  (word_embeds): Embedding(11243, 25)
  (lstm): LSTM(25, 25, bidirectional=True)
  (attention_layers): ModuleList(
    (0): Attention_layer(
      (K): Linear(in_features=50, out_features=50, bias=False)
      (Q): Linear(in_features=50, out_features=50, bias=False)
      (V): Linear(in_features=50, out_features=50, bias=False)
      (K_dropout): Dropout(p=0.0, inplace=False)
      (Q_dropout): Dropout(p=0.0, inplace=False)
      (V_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (hidden2tag): Linear(in_features=50, out_features=12, bias=True)
  (ce): CrossEntropyLoss()
)

Optimizer Setup: 
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

------ Start Training, Testing, Evaluating Process | Total Epoch: 2 ------
Epoch:1, Training loss: 34726.24, train acc: 0.6750, train f1: 0.5440, val loss: 10215.26, val acc: 0.6772, val f1: 0.5469, time: 73.77s
Epoch:2, Training loss: 29